# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guntakal junction,15.1667,77.3833,24.69,74,88,5.86,IN,1726168329
1,1,vernon,50.2581,-119.2691,19.96,54,63,0.89,CA,1726168330
2,2,sao gabriel da cachoeira,-0.1303,-67.0892,37.30,32,88,1.34,BR,1726168332
3,3,waitangi,-43.9535,-176.5597,11.57,95,100,3.13,NZ,1726168333
4,4,talnakh,69.4865,88.3972,7.47,95,100,4.58,RU,1726168334


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp'] < 28) & (city_data_df['Cloudiness'] < 20)
                                   & (city_data_df['Humidity'] < 70) & (city_data_df['Wind Speed'] < 3)]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,queenstown,-31.8976,26.8753,13.70,33,0,0.80,ZA,1726168336
88,88,ilulissat,69.2167,-51.1000,4.01,65,0,2.06,GL,1726168431
101,101,zahedan,29.4963,60.8629,16.95,13,0,2.06,IR,1726168446
112,112,geraldton,-28.7667,114.6000,16.23,59,0,2.06,AU,1726168458
126,126,luwingu,-10.2621,29.9271,21.72,30,0,2.63,ZM,1726168476
168,168,sretensk,52.2453,117.7093,1.26,57,0,1.28,RU,1726168527
294,294,arak,34.0917,49.6892,23.29,19,0,0.79,IR,1726168679
318,318,orapa,-21.2833,25.3667,24.69,21,0,1.27,BW,1726168707
319,319,saint-pierre,-21.3393,55.4781,21.82,68,0,2.57,RE,1726168708
324,324,shihezi,44.3000,86.0333,17.56,51,10,1.66,CN,1726168714


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df['Hotel Name'] = ''

# Display sample data
hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,queenstown,ZA,-31.8976,26.8753,33,
88,ilulissat,GL,69.2167,-51.1000,65,
101,zahedan,IR,29.4963,60.8629,13,
112,geraldton,AU,-28.7667,114.6000,59,
126,luwingu,ZM,-10.2621,29.9271,30,
168,sretensk,RU,52.2453,117.7093,57,
294,arak,IR,34.0917,49.6892,19,
318,orapa,BW,-21.2833,25.3667,21,
319,saint-pierre,RE,-21.3393,55.4781,68,
324,shihezi,CN,44.3000,86.0333,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "limit": limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotels_df.loc[index, "Lat"]
    longitude = hotels_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
queenstown - nearest hotel: Queens Hotel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
zahedan - nearest hotel: هتل صالح;بانک ملی
geraldton - nearest hotel: Ocean Centre Hotel
luwingu - nearest hotel: No hotel found
sretensk - nearest hotel: No hotel found
arak - nearest hotel: هتل زاگرس
orapa - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
shihezi - nearest hotel: 石河子宾馆
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
la sarre - nearest hotel: Motel le Bivouac
south windsor - nearest hotel: The Watson House B&B
kirkland lake - nearest hotel: No hotel found
jiagedaqi - nearest hotel: No hotel found
barkhan - nearest hotel: No hotel found
trancas - nearest hotel: Hotel
tukrah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
5,queenstown,ZA,-31.8976,26.8753,33,Queens Hotel
88,ilulissat,GL,69.2167,-51.1000,65,Best Western Plus Hotel Ilulissat
101,zahedan,IR,29.4963,60.8629,13,هتل صالح;بانک ملی
112,geraldton,AU,-28.7667,114.6000,59,Ocean Centre Hotel
126,luwingu,ZM,-10.2621,29.9271,30,No hotel found
168,sretensk,RU,52.2453,117.7093,57,No hotel found
294,arak,IR,34.0917,49.6892,19,هتل زاگرس
318,orapa,BW,-21.2833,25.3667,21,No hotel found
319,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
324,shihezi,CN,44.3000,86.0333,51,石河子宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)